In [1]:
import os  
#os.environ['THEANO_FLAGS'] = "device=gpu1"  
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32,lib.cnmem=1"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
  
#%pylab inline
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score


import theano
from keras.layers import Input, Dense
from keras.layers import Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.utils.np_utils import *

from sklearn.cross_validation import train_test_split

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
Using Theano backend.
/home/mrafi123/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
x_train = np.genfromtxt('x_train.out')
y_train = np.genfromtxt('y_train.out')
vx_train = np.genfromtxt('vx_train.out')
vy_train = np.genfromtxt('vy_train.out')
x_test = np.genfromtxt('x_test.out')
y_test = np.genfromtxt('y_test.out')

In [3]:
print (x_train.shape)
print (y_train.shape)
print (vx_train.shape)
print (vy_train.shape)
print (x_test.shape)
print (y_test.shape)

(50000, 784)
(50000,)
(10000, 784)
(10000,)
(10000, 784)
(10000,)


In [4]:
label_train=to_categorical(y_train)
label_valid=to_categorical(vy_train)
label_test= to_categorical(y_test)

/home/mrafi123/anaconda3/lib/python3.5/site-packages/keras/utils/np_utils.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y = np.zeros((len(y), nb_classes))
/home/mrafi123/anaconda3/lib/python3.5/site-packages/keras/utils/np_utils.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


In [5]:
print (x_train.shape)
print (label_train.shape)
print (vx_train.shape)
print (label_valid.shape)
print (x_test.shape)
print (label_test.shape)

(50000, 784)
(50000, 10)
(10000, 784)
(10000, 10)
(10000, 784)
(10000, 10)


In [6]:
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

In [7]:
# reshape data

#train_x_temp = x_train.reshape(-1, 28, 28, 1)
#val_x_temp = vx_train.reshape(-1, 28, 28, 1)
#test_x_temp=x_test.reshape(-1, 28, 28, 1)

train_x_temp = x_train.reshape(-1,1, 28, 28)
val_x_temp = vx_train.reshape(-1,1, 28, 28)
test_x_temp=x_test.reshape(-1,1, 28, 28)
print(train_x_temp.shape)
print(val_x_temp.shape)
print(test_x_temp.shape)


# define vars
input_shape = (784,)
input_reshape = (1,28, 28)

conv_num_filters = 5
conv_filter_size = 5

pool_size = (2, 2)

hidden_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128



(50000, 1, 28, 28)
(10000, 1, 28, 28)
(10000, 1, 28, 28)


In [12]:
# this is our input placeholder
input_img = Input(shape=(input_reshape))
x = Convolution2D(25, 5, 5, border_mode='same',activation='relu')(input_img) 
x = MaxPooling2D(pool_size=pool_size)(x)
x = Convolution2D(25, 5, 5, border_mode='same',activation='relu')(x)
x = MaxPooling2D(pool_size=pool_size)(x)
x = Convolution2D(25, 4, 4, border_mode='same',activation='relu')(x)
x = Flatten()(x)
x = Dense(output_dim=hidden_num_units, activation='relu')(x)
x = Dense(output_dim=output_num_units, activation='softmax')(x)
                  
model = Model(input=input_img, output=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model = Sequential([
 #InputLayer(input_shape=input_reshape),

 Convolution2D(25, 5, 5, border_mode='same',activation='relu',
              input_shape=input_reshape),
 MaxPooling2D(pool_size=pool_size),

 Convolution2D(25, 5, 5, border_mode='same',activation='relu'),
 MaxPooling2D(pool_size=pool_size),

 Convolution2D(25, 4, 4, border_mode='same',activation='relu'),

 Flatten(),

 Dense(output_dim=hidden_num_units, activation='relu'),

 Dense(output_dim=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 98.77%


In [13]:
trained_model_conv = model.fit(train_x_temp, label_train, 
                               nb_epoch=epochs, batch_size=batch_size, 
                               validation_data=(val_x_temp, label_valid))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 50s - loss: 0.2586 - acc: 0.9236 - val_loss: 0.0766 - val_acc: 0.9769
Epoch 2/5
50000/50000 [==============================] - 50s - loss: 0.0660 - acc: 0.9794 - val_loss: 0.0513 - val_acc: 0.9854
Epoch 3/5
50000/50000 [==============================] - 50s - loss: 0.0464 - acc: 0.9858 - val_loss: 0.0485 - val_acc: 0.9852
Epoch 4/5
50000/50000 [==============================] - 51s - loss: 0.0353 - acc: 0.9884 - val_loss: 0.0326 - val_acc: 0.9894
Epoch 5/5
50000/50000 [==============================] - 51s - loss: 0.0294 - acc: 0.9907 - val_loss: 0.0373 - val_acc: 0.9889


In [18]:
#pred = model.predict_classes(x_test)
# Keras Model nulike Sequential does not support predict_classes
y_pred = model.predict(test_x_temp)

In [19]:
y_pred

array([[  1.37583930e-10,   5.58858460e-10,   1.88497935e-07, ...,
          9.99999642e-01,   2.55897836e-09,   6.48487486e-08],
       [  5.16114085e-07,   5.66443805e-06,   9.99993682e-01, ...,
          1.09301712e-09,   6.77446081e-08,   1.17403632e-13],
       [  4.21860750e-06,   9.99842405e-01,   5.34309220e-06, ...,
          3.93980845e-05,   5.35651961e-05,   5.84102145e-06],
       ..., 
       [  1.46977558e-12,   1.06432818e-08,   6.45221723e-13, ...,
          6.07872792e-08,   5.40789920e-08,   1.46090642e-07],
       [  7.92561405e-10,   8.27558133e-10,   5.38640021e-10, ...,
          1.39895562e-08,   7.27315724e-04,   4.24962501e-08],
       [  8.79794513e-08,   3.17576625e-11,   7.23900245e-11, ...,
          4.25307567e-11,   2.16956608e-09,   3.29494126e-10]])

In [20]:
new_y_pred=[]
for i in range (len(y_pred)):
    new_y_pred.append([np.argmax(y_pred[i])])


In [21]:
new_y_pred=np.asarray(new_y_pred)
print (new_y_pred.shape)
new_y_pred=to_categorical(new_y_pred)
print (new_y_pred.shape)

(10000, 1)
(10000, 10)


In [22]:
new_y_pred

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [23]:
label_test

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
num=len(label_test)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(new_y_pred[i],label_test[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")

tested  10000 digits
correct:  9891 wrong:  109 error rate:  1.09 %
got correctly  98.91 %
